In [ ]:
"""総合演習2 マクロ系シミュレーション 第3回 例題1

2次元非圧縮流れのシミュレーション(渦度-流線関数法, 有限差分法)
"""

import sys
from typing import Final, Self

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from matplotlib import animation
from numba import njit

# ========== パラメータ ==========
LX: Final[float] = 10              # 部屋のx方向の長さ(m)
LY: Final[float] = LX              # 部屋のy方向の長さ(m)
RHO: Final[float] = 1              # 密度(kg/m^3)
NU: Final[float] = 0.01            # 動粘性係数(m^2/s)
V0: Final[float] = 0.2             # 空気の流れの速さ(m/s)

NX: Final[int] = 31                # x方向の格子点数
NY: Final[int] = NX                # y方向の格子点数
DT: Final[float] = 0.5             # 時間刻み幅(s)
T_END: Final[float] = 500          # シミュレーション時間(s)
DT_PLOT: Final[float] = T_END/50   # 等値線図を作成する時間間隔(s)

OMEGA_SOR: Final[float] = 1.8      # SOR法の加速パラメータ(0<OMEGA_SOR<2)
TOL: Final[float] = 10**(-3)       # SOR法の許容誤差(相対誤差)
MAX_ITER: Final[int] = 100         # SOR法の最大反復回数
# ==============================

# float型のNumPy配列の型エイリアス
FloatArray = np.typing.NDArray[np.float64]


class Variable:
    """格子点上の物理量を扱うクラス

    Attributes
    ----------
    value : FloatArray
        格子点上の物理量の値
    __name : str
        'grid_x', 'grid_y', 'psi', 'omg' のいずれか

    Notes
    -----
    インスタンスを生成する前に, set_class_variablesクラスメソッドを実行する必要あり
    Variableクラスの外で使用しない属性は, 変数名の最初にダンダー(__)をつけて隠蔽している
    """

    __lx: float
    __ly: float
    __nx: int
    __ny: int
    __dx: float
    __dy: float

    # set_class_variablesクラスメソッドを実行したかどうかを判定する真偽値
    __flag: bool = False

    @classmethod
    def set_class_variables(cls, lx: float, ly: float, nx: int, ny: int) -> None:
        """Variableクラスのクラス変数を設定する

        Parameters
        ----------
        lx : float
            溝のx方向の長さ
        ly : float
            溝のy方向の長さ
        nx : int
            x方向の格子点数
        ny : int
            y方向の格子点数
        """

        cls.__lx = lx
        cls.__ly = ly
        cls.__nx = nx
        cls.__ny = ny
        cls.__dx = lx / (nx-1)
        cls.__dy = ly / (ny-1)
        cls.__flag = True

    @classmethod
    def check_stable(cls, dt: float, v0: float, nu: float):
        """数値的安定性をチェックする
        
        数値的安定性の条件:
            時間刻み幅 < 移流または粘性拡散によって情報が格子点間を伝わるのにかかる時間

        Parameters
        ----------
        dt : float
            時間刻み幅
        v0 : float
            空気の流れの速さ
        nu : float
            動粘性係数
        """

        if not Variable.__flag:
            print('[ERROR] set_class_variablesクラスメソッドが実行されていません')
            sys.exit()

        dx: float = Variable.__dx
        dy: float = Variable.__dy
        dt_stable: float = min(1/(v0/dx+v0/dy), 1/((2*nu)*(1/(dx**2)+1/(dy**2))))
        if dt > dt_stable:
            print(f'[ERROR] 数値的安定性の条件(DT < {dt_stable})を満たしていません')
            sys.exit()

    def __init__(self, name: str, v0: float | None = None) -> None:
        """Variableクラスのイニシャライザ

        Parameters
        ----------
        name : str
            'grid_x', 'grid_y', 'psi', 'omg' のいずれか
        v0 : float | None, optional, default None
            空気の流れの速さ
        """

        if not Variable.__flag:
            print('[ERROR] set_class_variablesクラスメソッドが実行されていません')
            sys.exit()

        if name not in ['grid_x', 'grid_y', 'psi', 'omg']:
            print('[ERROR] Variableクラスのイニシャライザの引数が不適切です')
            sys.exit()
        self.__name: str = name

        if (name in ['psi', 'omg']) and (v0 is None):
            print('[ERROR] Variableクラスのイニシャライザの引数が不適切です')
            sys.exit()

        nx: int = Variable.__nx
        ny: int = Variable.__ny
        self.value: FloatArray = np.empty((nx, ny), dtype=np.float64)

        lx: float = Variable.__lx
        ly: float = Variable.__ly
        dx: float = Variable.__dx
        dy: float = Variable.__dy
        x: float
        y: float
        for ix in range(nx):
            x = dx * ix
            for iy in range(ny):
                y = dy * iy
                self.value[ix, iy] = self.__set_initial_condition(x, y, lx, ly, v0)

    def __set_initial_condition(self, x: float, y: float, lx: float, ly: float, v0: float | None) -> float:
        """(x,y)における(初期条件の)値を設定する

        Parameters
        ----------
        x : float
            溝の断面の左下を原点としたときのx
        y : float
            溝の断面の左下を原点としたときのy
        lx : float
            部屋のx方向の長さ
        ly : float
            部屋のy方向の長さ
        v0 : float | None
            空気の流れの速さ

        Returns
        -------
        float
            (x,y)における(初期条件の)値
        """

        if self.__name == 'grid_x':
            return x
        elif self.__name == 'grid_y':
            return y
        elif self.__name == 'psi':
            return -(v0 * np.sqrt(lx*ly) / np.pi) * (np.sin(np.pi * x / lx)**2) * (np.sin(np.pi * y / ly)**2)
        elif self.__name == 'omg':
            return (2 * np.pi * v0 / np.sqrt(lx*ly)) * (
                (ly/lx) * np.cos(2 * np.pi * x / lx) * (np.sin(np.pi * y / ly)**2)
                + (lx/ly) * (np.sin(np.pi * x / lx)**2) * np.cos(2 * np.pi * y / ly)
            )

    def set_boundary_condition(self, psi: Self | None = None) -> None:
        """境界条件を設定する

        左端: vx=0, vy=0  ->  psi=0, (∂psi/∂x)=0
        右端: vx=0, vy=0  ->  psi=0, (∂psi/∂x)=0
        下端: vx=0, vy=0  ->  psi=0, (∂psi/∂y)=0
        上端: vx=0, vy=0  ->  psi=0, (∂psi/∂y)=0

        Parameters
        ----------
        psi : Self | None, optional, default None
            Variableクラスのインスタンス(psi), omgの境界条件に使用
        """

        if self.__name == 'psi':

            self.value[0, :] = 0    # 左端
            self.value[-1, :] = 0   # 右端
            self.value[:, 0] = 0    # 下端
            self.value[:, -1] = 0   # 上端

        elif self.__name == 'omg':

            psi_name: str = ''
            if isinstance(psi, Variable):
                psi_name = psi.__name
            if (psi_name != 'psi'):
                print('[ERROR] omg.set_boundary_conditionメソッドの引数が不適切です')
                sys.exit()

            dx: float = Variable.__dx
            dy: float = Variable.__dy
            self.value[0, :] = -2 * (psi.value[1, :]-psi.value[0, :]) / (dx**2)      # 左端
            self.value[-1, :] = -2 * (psi.value[-2, :]-psi.value[-1, :]) / (dx**2)   # 右端
            self.value[:, 0] = -2 * (psi.value[:, 1]-psi.value[:, 0]) / (dy**2)      # 下端
            self.value[:, -1] = -2 * (psi.value[:, -2]-psi.value[:, -1]) / (dy**2)   # 上端

    def advection(self, psi: Self) -> FloatArray:
        """移流項を計算する (風上差分)

        Parameters
        ----------
        psi : Self
            Variableクラスのインスタンス(psi)

        Returns
        -------
        advection : FloatArray
            移流項
        """

        psi_name: str = ''
        if isinstance(psi, Variable):
            psi_name = psi.__name
        if psi_name != 'psi':
            print('[ERROR] advectionメソッドの引数が不適切です')
            sys.exit()

        nx: int = Variable.__nx
        ny: int = Variable.__ny
        advection: FloatArray = np.zeros((nx, ny), dtype=np.float64)

        dx: float = Variable.__dx
        dy: float = Variable.__dy
        vx: float
        vy: float
        for ix in range(1, nx-1):
            for iy in range(1, ny-1):
                vx = (psi.value[ix, iy+1] - psi.value[ix, iy-1]) / (2 * dy)
                vy = - (psi.value[ix+1, iy] - psi.value[ix-1, iy]) / (2 * dx)
                advection[ix, iy] = (
                    max(vx, 0) * (self.value[ix, iy] - self.value[ix-1, iy]) / dx
                    + min(vx, 0) * (self.value[ix+1, iy] - self.value[ix, iy]) / dx
                    + max(vy, 0) * (self.value[ix, iy] - self.value[ix, iy-1]) / dy
                    + min(vy, 0) * (self.value[ix, iy+1] - self.value[ix, iy]) / dy
                )
        return advection

    def laplacian(self) -> FloatArray:
        """ラプラシアンを計算する

        Returns
        -------
        laplacian : FloatArray
            ラプラシアン
        """

        nx: int = Variable.__nx
        ny: int = Variable.__ny
        laplacian: FloatArray = np.zeros((nx, ny), dtype=np.float64)

        dx: float = Variable.__dx
        dy: float = Variable.__dy
        for ix in range(1, nx-1):
            for iy in range(1, ny-1):
                laplacian[ix, iy] = (
                    (self.value[ix+1, iy] - 2 * self.value[ix, iy] + self.value[ix-1, iy]) / (dx**2)
                    + (self.value[ix, iy+1] - 2 * self.value[ix, iy] + self.value[ix, iy-1]) / (dy**2)
                )
        return laplacian

    def poisson_solver(self, source: FloatArray, omega_sor: float, tol: float, max_iter: int) -> None:
        """そのインスタンスを未知関数とするPoisson方程式を解く(SOR法)

        SOR(Successive Over-Relaxation, 逐次加速緩和)法:
            Gauss-Seidel法における値の更新を過剰に行うことで反復回数を減らし, 解への収束を速める

        Parameters
        ----------
        source : FloatArray
            Poisson方程式のソース項
        omega_sor : float
            SOR法の加速パラメータ(0<omega_sor<2)
        tol : float
            SOR法の許容誤差(相対誤差)
        max_iter : int
            SOR法の最大反復回数

        Notes
        -----
        このシミュレーション全体で一番計算コストがかかるのはPoisson方程式の求解(3重のforループあり)
        特にPythonではfor文が遅いので, ループ処理を行わずNumPyでベクトル化することが推奨されるが, SOR法をベクトル化するのは難しい
        なので, Numbaを用いて計算の高速化を図る
        ただし, Numbaではオブジェクトの属性アクセスが使えない(たぶん)ので, SOR法の反復1回分を別の関数としてVariableクラスの外に出している
        Numbaをオフにするにはsor_one_iter関数のデコレータ「@njit」をコメントアウトすればOK
        """

        dx: float = Variable.__dx
        dy: float = Variable.__dy
        max_err: float
        for _ in range(max_iter):
            self.value, max_err = sor_one_iter(self.value, source, dx, dy, omega_sor)
            self.set_boundary_condition()
            if max_err < tol:
                break


@njit
def sor_one_iter(solution: FloatArray, source: FloatArray, dx: float, dy: float, omega_sor: float) -> tuple[FloatArray, float]:
    """SOR法の反復1回分を計算する

    Parameters
    ----------
    solution : FloatArray
          Poisson方程式の未知関数
    source : FloatArray
          Poisson方程式のソース項
    dx : float
        x方向の格子点の間隔
    dy : float
        y方向の格子点の間隔
    omega_sor : float
        SOR法の加速パラメータ(0<omega_sor<2)

    Returns
    -------
    solution : FloatArray
        Poisson方程式の未知関数
    max_err : float
        最大誤差
    """

    nx: int
    ny: int
    nx, ny = solution.shape

    max_err: float = 0
    denominator: float = 2/(dx**2) + 2/(dy**2)

    old: float
    new: float
    err: float
    for ix in range(1, nx-1):
        for iy in range(1, ny-1):
            old = solution[ix, iy]
            new = (
                (solution[ix+1, iy] + solution[ix-1, iy]) / (dx**2)
                + (solution[ix, iy+1] + solution[ix, iy-1]) / (dy**2)
                - source[ix, iy]
            ) / denominator
            solution[ix, iy] = (1-omega_sor) * old + omega_sor * new

            if new != 0:
                err = abs((new - old) / new)
                max_err = max(max_err, err)

    return solution, max_err


def create_plot(frame: int, *fargs) -> tuple:
    """各タイムステップでの等値線図を作成する
    
    animation.FuncAnimationに使用する関数
    
    Parameters
    ----------
    frame : int
        フレーム番号
    fargs : tuple
        格子点のx座標, 格子点のy座標, 時刻, psiの結果, omgの結果を格納したタプル
    """

    grid_x: FloatArray
    grid_y: FloatArray
    results_t: list[float]
    results_psi: list[FloatArray]
    results_omg: list[FloatArray]
    grid_x, grid_y, results_t, results_psi, results_omg = fargs
    
    axes[0].cla()
    axes[1].cla()

    # psi の等値線図
    psi: FloatArray = results_psi[frame]
    vmin: float = np.min(results_psi)
    vmax: float = np.max(results_psi)
    im1 = axes[0].contourf(grid_x, grid_y, psi, vmin=vmin, vmax=vmax, levels=20, cmap='jet')
    axes[0].contour(im1, colors='k', linewidths=0.5)
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].set_title('Stream Function (psi)')
    axes[0].set_aspect('equal')

    # omg の等値線図
    omg: FloatArray = results_omg[frame]
    vmin = np.min(results_omg)
    vmax = np.max(results_omg)
    im2 = axes[1].contourf(grid_x, grid_y, omg, vmin=vmin, vmax=vmax, levels=20, cmap='jet')
    axes[1].contour(im2, colors='k', linewidths=0.5)
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')
    axes[1].set_title('Vorticity (omg)')
    axes[1].set_aspect('equal')

    figure.suptitle(f't = {results_t[frame]:.1f} sec')

    return im1, im2


if __name__ == '__main__':

    Variable.set_class_variables(LX, LY, NX, NY)
    Variable.check_stable(DT, V0, NU)

    # Variableクラスのインスタンスとして変数を準備
    grid_x: Variable = Variable('grid_x')
    grid_y: Variable = Variable('grid_y')
    psi: Variable = Variable('psi', V0)
    omg: Variable = Variable('omg', V0)

    t: float = 0
    it: int = 0

    # アニメーションの準備
    figure, axes = plt.subplots(1, 2, figsize=(10, 5))
    results_t: list[float] = [t]
    results_psi: list[FloatArray] = [psi.value.copy()]
    results_omg: list[FloatArray] = [omg.value.copy()]

    while t < T_END:

        # 方程式にしたがって時間発展(Euler法)
        omg.value += (-omg.advection(psi) + NU * omg.laplacian()) * DT
        psi.poisson_solver(-omg.value, OMEGA_SOR, TOL, MAX_ITER)
        omg.set_boundary_condition(psi)
        t += DT

        # アニメーション用に結果を保存
        if it % int(DT_PLOT / DT) == 0:
            results_t.append(t)
            results_psi.append(psi.value.copy())
            results_omg.append(omg.value.copy())
        it += 1

    # アニメーションを作成
    im1, im2 = create_plot(0, grid_x.value, grid_y.value, results_t, results_psi, results_omg)
    figure.colorbar(im1, ax=axes[0])
    figure.colorbar(im2, ax=axes[1])
    figure.tight_layout()
    anim = animation.FuncAnimation(figure, create_plot, range(len(results_t)),
                                   fargs=(grid_x.value, grid_y.value, results_t, results_psi, results_omg))
    display(HTML(anim.to_jshtml()))
    plt.close()